In [16]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Dense, Flatten, Activation
from keras.utils import np_utils
from keras import models

# Let's load our data set straight from Keras to save time.
# It's already split into training and testing, too!
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# We're going to resize everything to 28 by 28, so lets set that size
img_width = 28
img_height = 28

#  Now we need to reshape our training and testing data.
#  We'll explicitly set the data type as a 32bit float
# Then we'll divide everything by 255.
X_train = X_train.astype("float32")
X_train /= 255.
X_test = X_test.astype("float32")
X_test /= 255.

# Now we'll finish data preparation by reshaping our image data
X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 1)
X_test = X_test.reshape(X_test.shape[0], img_width, img_height, 1)


# Now we need to reshape our labels for one-hot encoding.
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

# #  We're done with data manipulation--let's set up our model!
# #  First, we'll create our model object.
# model = Sequential()
# #  Now, we'll add a 2D Convolutional Neural Network.
# model.add(Convolution2D(8, 5, 5, input_shape=(img_width, img_height, 1), activation="relu"))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.2))
# model.add(Flatten())
# model.add(Dense(32, activation="relu"))
# model.add(Dense(num_classes, activation="softmax"))
# # TODO:  Port code to python2 so that it will work with CoreML
# # TODO:  Add comments explaining methodology for rest of model
# # TODO:  Export model with CoreML library.
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
# model.summary()
# model.fit(X_train, y_train, batch_size=128, nb_epoch=2, validation_data=(X_test, y_test))
# models.save_model(model, 'keras_mnist.h5')

In [22]:
model = Sequential()

model.add(Convolution2D(8, 5, 5, border_mode='valid', input_shape=(img_width, img_height, 1)))
model.add(Activation('relu'))
model.add(Convolution2D(8, 5, 5))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=128, nb_epoch=12,
          verbose=1, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])
models.save_model(model,'mnist_cnn_keras_122.h5')

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 31s - loss: 0.5022 - acc: 0.8404 - val_loss: 0.1213 - val_acc: 0.9640
Epoch 2/12
60000/60000 [==============================] - 30s - loss: 0.2014 - acc: 0.9392 - val_loss: 0.0917 - val_acc: 0.9708
Epoch 3/12
60000/60000 [==============================] - 30s - loss: 0.1678 - acc: 0.9507 - val_loss: 0.0761 - val_acc: 0.9755
Epoch 4/12
60000/60000 [==============================] - 30s - loss: 0.1454 - acc: 0.9565 - val_loss: 0.0671 - val_acc: 0.9779
Epoch 5/12
60000/60000 [==============================] - 31s - loss: 0.1286 - acc: 0.9626 - val_loss: 0.0616 - val_acc: 0.9803
Epoch 6/12
60000/60000 [==============================] - 29s - loss: 0.1179 - acc: 0.9647 - val_loss: 0.0551 - val_acc: 0.9828
Epoch 7/12
60000/60000 [==============================] - 30s - loss: 0.1084 - acc: 0.9673 - val_loss: 0.0509 - val_acc: 0.9836
Epoch 8/12
60000/60000 [==============================

In [23]:
import coremltools

In [24]:
coreml_model = coremltools.converters.keras.convert('keras_mnist.h5')

0 : convolution2d_input_16, <keras.engine.topology.InputLayer object at 0x1290998d0>
1 : convolution2d_3, <keras.layers.convolutional.Convolution2D object at 0x1290997d0>
2 : convolution2d_3__activation__, <keras.layers.core.Activation object at 0x129817f90>
3 : maxpooling2d_3, <keras.layers.pooling.MaxPooling2D object at 0x1290a55d0>
4 : flatten_3, <keras.layers.core.Flatten object at 0x129099d90>
5 : dense_5, <keras.layers.core.Dense object at 0x1290e5a90>
6 : dense_5__activation__, <keras.layers.core.Activation object at 0x1297abe10>
7 : dense_6, <keras.layers.core.Dense object at 0x129167690>
8 : dense_6__activation__, <keras.layers.core.Activation object at 0x1297abc90>


In [25]:
coreml_model.save('keras_mnist.mlmodel')

In [26]:
coreml_model

input {
  name: "input1"
  type {
    multiArrayType {
      shape: 1
      shape: 28
      shape: 28
      dataType: DOUBLE
    }
  }
}
output {
  name: "output1"
  type {
    multiArrayType {
      shape: 10
      dataType: DOUBLE
    }
  }
}